In [1]:
import sys
sys.path.append("../")
sys.path.append('C:\Users\James Pino\PycharmProjects\Magine')
sys.path.append('C:\Users\James Pino\PycharmProjects\magine_gui_app')

In [2]:
from IPython.display import display, HTML
%matplotlib inline
import pandas as pd
import networkx as nx
import magine.ontology.enrichment_tools as et
import magine.networks.visualization.notebook_tools as nt
from magine.plotting.wordcloud_tools import word_cloud_from_array

<IPython.core.display.Javascript object>

First, lets load the molecular network and the enrichment output from the previous runs. 

In [3]:
enrichment_array = pd.read_csv('Data/all_cisplatin_out.csv.gz', index_col=0)
network = nx.read_gpickle('Data/cisplatin_based_network.p')

This enrichment dataframe contains all the enrichment data from the various databases.

In [4]:
display(enrichment_array.head(10))

,term_name,combined_score,adj_p_value,rank,genes,n_genes,db,category,sample_id
0,"mRNA splicing, via spliceosome",244.428316,4.004356e-25,1,"ALYREF,DDX39A,DDX39B,DDX5,DHX9,ELAVL1,FIP1L1,FUS,HNRNPA0,HNRNPA1,HNRNPA2B1,HNRNPA3,HNRNPC,HNRNPD...",54,GO_Biological_Process_2017,proteomics_both,01hr
1,SRP-dependent cotranslational protein targeting to membrane,163.857942,8.882552e-17,2,"RPL11,RPL13,RPL13A,RPL14,RPL15,RPL17,RPL18A,RPL19,RPL21,RPL22,RPL23A,RPL29,RPL3,RPL36,RPL37A,RPL...",28,GO_Biological_Process_2017,proteomics_both,01hr
2,"nuclear-transcribed mRNA catabolic process, nonsense-mediated decay",158.589542,6.926429e-16,3,"PABPC1,RBM8A,RPL11,RPL13,RPL13A,RPL14,RPL15,RPL17,RPL18A,RPL19,RPL21,RPL22,RPL23A,RPL29,RPL3,RPL...",30,GO_Biological_Process_2017,proteomics_both,01hr
3,rRNA processing,157.751396,9.500549e-18,4,"DKC1,FBL,NOL8,NOLC1,NOP56,RPL11,RPL13,RPL13A,RPL14,RPL15,RPL17,RPL18A,RPL19,RPL21,RPL22,RPL23A,R...",40,GO_Biological_Process_2017,proteomics_both,01hr
4,translational initiation,131.751148,2.181663e-17,5,"EIF2S2,EIF2S3,EIF3B,PABPC1,RPL11,RPL13,RPL13A,RPL14,RPL15,RPL17,RPL18A,RPL19,RPL21,RPL22,RPL23A,...",32,GO_Biological_Process_2017,proteomics_both,01hr
5,viral transcription,130.939337,7.045307e-18,6,"NUP188,NUP88,NUP98,RPL11,RPL13,RPL13A,RPL14,RPL15,RPL17,RPL18A,RPL19,RPL21,RPL22,RPL23A,RPL29,RP...",32,GO_Biological_Process_2017,proteomics_both,01hr
6,translation,116.719681,2.636988e-14,7,"EIF4EBP2,MRPS33,PABPC4,RPL11,RPL13,RPL13A,RPL14,RPL15,RPL17,RPL18A,RPL19,RPL21,RPL22,RPL23A,RPL2...",33,GO_Biological_Process_2017,proteomics_both,01hr
7,mRNA processing,97.760595,1.501360e-15,8,"ALKBH5,CPSF6,HNRNPA0,HNRNPA1,HNRNPA2B1,KIN,NONO,PRPF3,PRPF8,PTBP1,SF3A1,SFPQ,SNRNP70,SON,SRSF1,S...",21,GO_Biological_Process_2017,proteomics_both,01hr
8,RNA export from nucleus,76.122617,3.366579e-12,9,"ALYREF,DDX39A,DDX39B,HNRNPA1,NUP188,NUP98,RANBP3,RBM8A,SARNP,SRSF1,SRSF3,SRSF4,SRSF5,SRSF6,SRSF7...",20,GO_Biological_Process_2017,proteomics_both,01hr
9,termination of RNA polymerase II transcription,74.621748,3.137348e-11,10,"ALYREF,DDX39A,DDX39B,FIP1L1,RBM8A,SARNP,SNRPB,SNRPD3,SNRPE,SNRPF,SRSF1,SRSF3,SRSF4,SRSF5,SRSF6,S...",20,GO_Biological_Process_2017,proteomics_both,01hr


For this example, we have proteomics and RNAseq data. We assume they are on different time scales, we analyze them separetly.

In [5]:
proteomics = enrichment_array[enrichment_array['category'].str.contains('proteomics')].copy()
rnaseq = enrichment_array[enrichment_array['category'].str.contains('rna')].copy()

For this example, we just want to look at "biological processes" descriptions, so we will limit our analysis to only databases with this type of information

In [6]:
print(proteomics['db'].unique())

process_dbs = [
        'GO_Biological_Process_2017',
        'Humancyc_2016',
        'Reactome_2016',
        'KEGG_2016',
        'NCI-Nature_2016',
        'Panther_2016',
        'WikiPathways_2016',
]


['GO_Biological_Process_2017' 'GO_Molecular_Function_2017'
 'GO_Cellular_Component_2017' 'KEGG_2016' 'NCI-Nature_2016' 'Panther_2016'
 'WikiPathways_2016' 'BioCarta_2016' 'Humancyc_2016' 'Reactome_2016'
 'KEA_2015' 'ChEA_2016' 'DrugMatrix' 'Drug_Perturbations_from_GEO_2014']


In [7]:
time_1_hour_prot = et.filter_dataframe(proteomics, p_value=0.05, combined_score=0.0, category='proteomics_up', sample_id='01hr', db=process_dbs)
display(time_1_hour_prot)

,term_name,combined_score,adj_p_value,rank,genes,n_genes,db,category,sample_id
112198,"negative regulation of transcription, DNA-templated",68.261978,0.001601,1,"ARID5B,BCLAF1,BRCA1,CALR,CBY1,CHD8,ENO1,HDAC1,ILF3,KHDRBS1,LIMS1,LRRFIP1,MYB,RBM15,RSF1,SHC1,SPE...",22,GO_Biological_Process_2017,proteomics_up,01hr
114511,Cell Cycle_Homo sapiens_R-HSA-1640170,36.507320,0.000267,1,"ACD,AKAP9,BRCA1,CDC16,CDC20,CDC7,CLASP2,DCTN1,HAUS8,HDAC1,HSPA2,KNTC1,LMNA,MAPK3,MYC,NPM1,NUP188...",28,Reactome_2016,proteomics_up,01hr
114512,Interleukin-2 signaling_Homo sapiens_R-HSA-451927,30.220851,0.001586,2,"AKAP9,BRAF,CNKSR2,CUL3,HAVCR2,INPPL1,IRS2,MAPK3,MARK3,PTPRA,RASGRF2,SHC1,SPTBN1,SPTBN2,TLN1,VWF",16,Reactome_2016,proteomics_up,01hr
112200,resolution of meiotic recombination intermediates,29.557281,0.016485,3,"ERCC4,SLX4,TOP2A,TOP2B",4,GO_Biological_Process_2017,proteomics_up,01hr
114513,"Interleukin-3, 5 and GM-CSF signaling_Homo sapiens_R-HSA-512988",29.303950,0.001586,3,"AKAP9,BRAF,CNKSR2,CUL3,INPPL1,IRS2,MAPK3,MARK3,PTPRA,RASGRF2,SHC1,SPTBN1,SPTBN2,TLN1,VWF,YWHAZ",16,Reactome_2016,proteomics_up,01hr
114514,"Cell Cycle, Mitotic_Homo sapiens_R-HSA-69278",27.727089,0.001586,4,"AKAP9,CDC16,CDC20,CDC7,CLASP2,DCTN1,HAUS8,HDAC1,KNTC1,LMNA,MAPK3,MYC,NUP188,NUP88,PDS5B,PLK1,PRI...",22,Reactome_2016,proteomics_up,01hr
114515,Interleukin receptor SHC signaling_Homo sapiens_R-HSA-912526,27.242088,0.001586,5,"AKAP9,BRAF,CNKSR2,CUL3,INPPL1,IRS2,MAPK3,MARK3,PTPRA,RASGRF2,SHC1,SPTBN1,SPTBN2,TLN1,VWF",15,Reactome_2016,proteomics_up,01hr
114516,MAPK family signaling cascades_Homo sapiens_R-HSA-5683057,26.440058,0.001586,6,"AKAP9,BRAF,CNKSR2,CUL3,DNAJB1,IRS2,MAPK3,MARK3,MYC,PTPRA,RASGRF2,SHC1,SPTBN1,SPTBN2,TLN1,VWF",16,Reactome_2016,proteomics_up,01hr
114517,Signaling by FGFR2_Homo sapiens_R-HSA-5654738,25.922377,0.001586,7,"AKAP9,BRAF,CNKSR2,CUL3,HNRNPA1,HNRNPM,INSR,IRS2,MAPK3,MAPKAP1,MARK3,PTPRA,RASGRF2,SHC1,SPTBN1,SP...",18,Reactome_2016,proteomics_up,01hr
114518,Insulin receptor signalling cascade_Homo sapiens_R-HSA-74751,25.791980,0.001586,8,"AKAP9,BRAF,CNKSR2,CUL3,INSR,IRS2,MAPK3,MARK3,PIK3R4,PTPRA,RASGRF2,SHC1,SPTBN1,SPTBN2,TLN1,VWF",16,Reactome_2016,proteomics_up,01hr


Since terms across databases might be redundant ("Interleukin-3, 5 and GM-CSF signaling_Homo sapiens_R-HSA-512988" and "Interleukin receptor SHC signaling_Homo sapiens_R-HSA-912526" have nearly a full overlap of genes), we want to eliminate duplicate terms and focus on the most enriched. For that we use the Jaccard Index. It is implmented in the find_similar_terms function in enrichment_tools (et).

In [8]:
filtered = et.find_similar_terms(time_1_hour_prot)
display(filtered)

Number of rows went from 89 to 32


,term_name,combined_score,adj_p_value,rank,genes,n_genes,db,category,sample_id
112198,"negative regulation of transcription, DNA-templated",68.261978,0.001601,1,"ARID5B,BCLAF1,BRCA1,CALR,CBY1,CHD8,ENO1,HDAC1,ILF3,KHDRBS1,LIMS1,LRRFIP1,MYB,RBM15,RSF1,SHC1,SPE...",22,GO_Biological_Process_2017,proteomics_up,01hr
114511,Cell Cycle_Homo sapiens_R-HSA-1640170,36.507320,0.000267,1,"ACD,AKAP9,BRCA1,CDC16,CDC20,CDC7,CLASP2,DCTN1,HAUS8,HDAC1,HSPA2,KNTC1,LMNA,MAPK3,MYC,NPM1,NUP188...",28,Reactome_2016,proteomics_up,01hr
114512,Interleukin-2 signaling_Homo sapiens_R-HSA-451927,30.220851,0.001586,2,"AKAP9,BRAF,CNKSR2,CUL3,HAVCR2,INPPL1,IRS2,MAPK3,MARK3,PTPRA,RASGRF2,SHC1,SPTBN1,SPTBN2,TLN1,VWF",16,Reactome_2016,proteomics_up,01hr
112200,resolution of meiotic recombination intermediates,29.557281,0.016485,3,"ERCC4,SLX4,TOP2A,TOP2B",4,GO_Biological_Process_2017,proteomics_up,01hr
114517,Signaling by FGFR2_Homo sapiens_R-HSA-5654738,25.922377,0.001586,7,"AKAP9,BRAF,CNKSR2,CUL3,HNRNPA1,HNRNPM,INSR,IRS2,MAPK3,MAPKAP1,MARK3,PTPRA,RASGRF2,SHC1,SPTBN1,SP...",18,Reactome_2016,proteomics_up,01hr
114519,Signalling by NGF_Homo sapiens_R-HSA-166520,25.783068,0.001586,9,"AKAP13,AKAP9,ARHGEF16,BRAF,CNKSR2,CUL3,HDAC1,INSR,IRS2,MAPK3,MAPKAP1,MARK3,MEF2C,PTPRA,RASGRF2,S...",21,Reactome_2016,proteomics_up,01hr
112204,mitotic cytokinesis,24.798241,0.030953,7,"APC,CEP55,PDCD6IP,PLK1,SPTBN1",5,GO_Biological_Process_2017,proteomics_up,01hr
114533,MAP2K and MAPK activation_Homo sapiens_R-HSA-5674135,22.681718,0.001586,23,"BRAF,CNKSR2,MAPK3,MARK3,TLN1,VWF",6,Reactome_2016,proteomics_up,01hr
114092,XPodNet - protein-protein interactions in the podocyte expanded by STRING_Mus musculus_WP2309,22.460249,0.011088,1,"APC,ARPC1A,BRAF,CAPZA1,CDH7,CLNK,EIF4ENIF1,IGFBP2,INPPL1,INSR,IRS2,KHDRBS1,KLC2,LASP1,LIMS1,LRP1...",30,WikiPathways_2016,proteomics_up,01hr
112205,chloride transport,22.310453,0.029255,8,"ANO4,ANO5,ANO6,CLIC4,SLC26A8",5,GO_Biological_Process_2017,proteomics_up,01hr


Now we can explore the top hits, which has been slimmed from 89 to 33 terms. Generally this is where the expert knowledge comes in. However, a quick search with each term and search terms of you molecule of interest tend to be useful. 

The first hit is 'negative regulation of transcription', which means that something caused genes not to be transcribed. Cisplatin causes DNA damage, thus negative regulaton of transcription makes sense. So does top hit 2, 'Cell Cycle_Homo sapiens_R-HSA-1640170'. 

A quick search for 'Interleukin-2 signaling' and 'Cisplatin' __[link](https://www.google.com/search?rlz=1C1CHBD_enUS721US721&ei=KzNeWuCxBsfq_AaSgYuYDQ&q=Interleukin-2+signaling+cisplatin&oq=Interleukin-2+signaling+cisplatin&gs_l=psy-ab.3..35i39k1.8097.9052.0.9196.10.10.0.0.0.0.145.897.7j3.10.0....0...1c.1.64.psy-ab..3.2.218....0.TInUjcZY740)__ returns a paper titled "Cisplatin at clinically relevant concentrations enhances interleukin-2 synthesis by human primary blood lymphocytes." __[link](https://www.ncbi.nlm.nih.gov/pubmed/10211553)__


We can link the two together to see how once might regulate the other by looking at the molecular interactions.

In [15]:
shorten_names = {
    'Cell Cycle_Homo sapiens_R-HSA-1640170':'Cell Cycle',
    'DNA Repair_Homo sapiens_R-HSA-73894' : 'DNA Repair',
    'chloride transport ' : 'chloride transport',
  'Interleukin-2 signaling_Homo sapiens_R-HSA-451927': 'IL2',
     'resolution of meiotic recombination intermediates ': 'Meiotic Recombination'
                }
renamed = proteomics.copy()
renamed['term_name'] = renamed['term_name'].replace(shorten_names)
term_net, mol_net = nt.create_subnetwork(shorten_names.values(), renamed, network, '1hr', cytoscape_js=False)

('DNA Repair', 79)
('IL2', 72)
('chloride transport', 7)
('Cell Cycle', 200)
('Meiotic Recombination', 4)
Looking for direct edges
No edges between chloride transport and Meiotic Recombination
No edges between chloride transport and IL2


In [18]:
nt.display_graph(term_net)
# nt.render_graph(term_net)

In [19]:
# nt.display_graph(mol_net,True)
nt.render_graph(mol_net)

Now we will explore the enrichment per time point looking at proteomics data that is up-regulated.

In [ ]:
nt.find_neighbors(g=mol_net, start='EP300', up_stream=True, down_stream=False)

In [ ]:
prot_enriched, words_array = word_cloud_from_array(enrichment_array, 'proteomics_up', ['01hr','06hr','24hr','48hr'],
                                             database_list=process_dbs)

Now, we will go through each time point and see what terms are reoccuring in the enrichment results.

In [ ]:
display(words_array.sort_values('01hr', ascending=False).head(20))

In [ ]:
# added terms for 1hr
keywords_1 = [
    'ner',
    'nucleotide excision',
    'cell cycle',
    'frs',
    'egfr',
    'insulin',
    'transcription',
    'mapk'
]

In [ ]:
time_6_hour_prot = et.filter_dataframe(proteomics, p_value=0.05, combined_score=0.0, category='proteomics_up', sample_id='06hr', db=process_dbs)
slimmed = et.find_similar_terms(time_6_hour_prot)
display(slimmed.head(25))

In [ ]:
time_24_hour_prot = et.filter_dataframe(proteomics, p_value=0.05, combined_score=0.0, category='proteomics_up', sample_id='24hr', db=process_dbs)
slimmed = et.find_similar_terms(time_24_hour_prot)
display(slimmed.head(25))

In [ ]:
shorten_names = {
                 'Cell Cycle_Homo sapiens_R-HSA-1640170': 'Cell Cycle',
                 'DNA Repair_Homo sapiens_R-HSA-73894': 'DNA Repair',
                 'positive regulation of apoptotic process ' : 'Apoptotic process',
                }
renamed = proteomics.copy()
renamed['term_name'] = renamed['term_name'].replace(shorten_names)
term_net, mol_net = create_subnetwork(shorten_names.values(), renamed, network, '1hr', cytoscape_js=False)

In [ ]:
nt.display_graph(mol_net,add_parent=True )

In [ ]:
nt.display_graph(term_net)

Now we can use these words to filter our enrichment array.

In [ ]:
filtered = et.filter_based_on_words(prot_enriched[0], keywords_1)
filtered = et.find_similar_terms(filtered)
display(filtered.head(25))
print("There are {} terms with selected keywords".format(filtered.shape[0]))

Lets start storing terms to look at.

In [ ]:
interesting_terms_1 = []

interesting_terms_1.append('Cell Cycle_Homo sapiens_R-HSA-1640170')
interesting_terms_1.append('MAPK family signaling cascades_Homo sapiens_R-HSA-5683057')
interesting_terms_1.append('Transcription-Coupled Nucleotide Excision Repair (TC-NER)_Homo sapiens_R-HSA-6781827')

This is where the network we generated comes into play. We load it, then use the function "create_subnetwork" to create a network connecting the two terms we are interested in.

In [ ]:
nt.create_subnetwork(interesting_terms_1, proteomics, network, '1hr', cytoscape_js=True)

In [ ]:
display(words_array.sort_values('06hr', ascending=False).head(45))

In [ ]:
#  added terms from 6hr
keywords_6 = [
#     'tp53',  # 7 hits
#     'dna damage', # 4 hits
#     'cell cycle', # 5 hits
#     'cell death', # 3 hits
#     'damage response',  # 3 hits
#     'apoptotic',  # 3 hits
    'response',
    'transcription'
]

In [ ]:
filtered = filter_based_on_words(prot_enriched[1], keywords_6)
display(filtered)
print("There are {} terms with selected keywords".format(filtered.shape[0]))

In [ ]:
interesting_terms_2 = []

interesting_terms_2.append('Transcriptional Regulation by TP53_Homo sapiens_R-HSA-3700989')
interesting_terms_2.append('Cell Cycle Checkpoints_Homo sapiens_R-HSA-69620')
interesting_terms_2.append('DNA Damage Response_Homo sapiens_WP707')

In [ ]:
nt.create_subnetwork(interesting_terms_2, proteomics, network, '6hr', cytoscape_js=True)

In [ ]:
display(words_array.sort_values('24hr', ascending=False).head(45))

In [ ]:
keywords_24 = [
    'dna damage', # 12 hits
    'apoptosis', # 8 hits
    'damage response',  # 7 hits
    'cell death',
    'growth factor',
]

filtered = filter_based_on_words(prot_enriched[2], keywords_24)
display(filtered)
print("There are {} terms with selected keywords".format(filtered.shape[0]))

In [ ]:
interesting_terms_3 = []

interesting_terms_3.append('Caspase Cascade in Apoptosis_Homo sapiens_b9d3ef2e-618f-11e5-8ac5-06603eb7f303')
interesting_terms_3.append('Apoptosis_Homo sapiens_R-HSA-109581')
create_subnetwork(interesting_terms_3, proteomics, network, '24hr', cytoscape_js=True)

In [ ]:
keywords_48 = [
    'apoptotic', # 6 hits
    'apoptosis', # 5 hits
    'cell proliferation ',  # 4 hits
    'cell cycle',   # 4 hits
    'damage response',  #
]

filtered = filter_based_on_words(prot_enriched[3], keywords_48)
display(filtered)
print("There are {} terms with selected keywords".format(filtered.shape[0]))

In [ ]:
interesting_terms_4 = []

interesting_terms_4.append('negative regulation of apoptotic process ')
interesting_terms_4.append('positive regulation of cell proliferation ')
interesting_terms_4.append('positive regulation of apoptotic process ')
nt.create_subnetwork(interesting_terms_4, proteomics, network, '48hr', cytoscape_js=True)

In [ ]:

all_interesting = interesting_terms_1 + interesting_terms_2 + interesting_terms_3 + interesting_terms_4

nt.create_subnetwork(all_interesting, proteomics, network, 'all_times', cytoscape_js=False)